In [1]:
import sklearn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager
from datetime import datetime, timedelta
%matplotlib inline

f_path = "C:\Windows\Fonts\malgun.ttf"
font_manager.FontProperties(fname=f_path).get_name()

from matplotlib import rc
rc('font', family='Malgun Gothic')
%matplotlib inline

### 1. 데이터가져오기 및 날짜표기 맞춰주기
- 데이터파일
 - ls_war2.csv : 전쟁목록
 - industry_code.csv : 미국상장기업 심볼리스트
 - us_historical_stock_price.csv : 시가총액 100대기업의 2000년이후 일별 주가변동자료

- 전쟁목록 및 일별주가변동 자료에 대한 날짜 형변형

In [2]:
# 전쟁목록 데이터 불러오기
ls_war = pd.read_csv("datas/ls_war2.csv", sep='\t')
ls_war = ls_war.drop(["Unnamed: 0", "종료시점", "위치"], axis=1)
ls_war.head()

,발발시점,갈등이름,갈등종류,전쟁규모,참전국수,전쟁수
0,2001.9,쿠르드이슬람교분쟁,민족,71000.0,3,1
1,2001.9.11,테러와의전쟁,테러,NaN,48,1
2,2002.1.15,영원한자유작전-필리핀,테러,115800.0,2,1
3,2002.10.7,영원한자유작전-아프리카의뿔,테러,NaN,38,1
4,2006,탈레반반란,테러,416000.0,15,1


In [3]:
# 전쟁 발발시점 불러오기
ls_war.loc[1, "발발시점"]

'2001.9.11'

In [4]:
# 전쟁목록에서 날짜 형태가 맞지 않은 row 삭제
ls_war2 = ls_war.drop(index=[0])
ls_war2 = ls_war2.drop(index=[4])
ls_war2.head()

,발발시점,갈등이름,갈등종류,전쟁규모,참전국수,전쟁수
1,2001.9.11,테러와의전쟁,테러,NaN,48,1
2,2002.1.15,영원한자유작전-필리핀,테러,115800.0,2,1
3,2002.10.7,영원한자유작전-아프리카의뿔,테러,NaN,38,1
5,2003.3.20,이라크전쟁,테러,NaN,41,1
6,2004.4.1,이란-PJAK충돌,민족,17000.0,3,1


In [5]:
# 인더스트리 파일 불러오기
industry = pd.read_csv("datas/industry_code.csv")
industry.head(2)

,Unnamed: 0,Symbol,Name,Industry,IndustryCode
0,0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",571060.0
1,1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,572010.0


In [6]:
# 미국기업 주가 히스토리 파일 불러오기
real_df = pd.read_csv("datas/us_historical_stock_prices.csv")
real_df.head(2)

,Unnamed: 0,Date,Close,Open,High,Low,Volume,Change,Symbol
0,0,2000-01-03,1.00,0.94,1.00,0.91,535820000.0,0.087,AAPL
1,1,2000-01-04,0.92,0.97,0.99,0.90,511730000.0,-0.080,AAPL


In [7]:
# Date타입변환
ls_war2['발발시점'] = pd.to_datetime(ls_war2['발발시점'], format='%Y-%m-%d')

In [8]:
# Date타입변환
real_df['Date'] = pd.to_datetime(real_df['Date'], format='%Y-%m-%d')
real_df.head(2)

,Unnamed: 0,Date,Close,Open,High,Low,Volume,Change,Symbol
0,0,2000-01-03,1.00,0.94,1.00,0.91,535820000.0,0.087,AAPL
1,1,2000-01-04,0.92,0.97,0.99,0.90,511730000.0,-0.080,AAPL


In [9]:
#Date타입 변환 확인
ls_war2 = ls_war2.reset_index(drop=True)
ls_war2.head(10)

,발발시점,갈등이름,갈등종류,전쟁규모,참전국수,전쟁수
0,2001-09-11,테러와의전쟁,테러,NaN,48,1
1,2002-01-15,영원한자유작전-필리핀,테러,115800.0,2,1
2,2002-10-07,영원한자유작전-아프리카의뿔,테러,NaN,38,1
3,2003-03-20,이라크전쟁,테러,NaN,41,1
4,2004-04-01,이란-PJAK충돌,민족,17000.0,3,1
5,2004-04-18,예멘의 Houthi 반란,내란,32000.0,7,1
6,2005-08-27,파라과이인민군반란,내란,3650.0,3,1
7,2006-01-25,파타-하마스 분쟁,테러,55000.0,3,1
8,2006-06-12,2006년레바논전쟁,테러,10500.0,2,1
9,2006-02-22,이라크내전,내란,748000.0,5,1


In [10]:
#Date타입 변환 확인
df_3 = industry.drop(['Unnamed: 0', 'IndustryCode'], axis=1)
df_3.head(2)

,Symbol,Name,Industry
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품"
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스


### 2. 3개의 파일 Merge한 DataFrame 만들기
 - 인더스트리의 Symbol에 전쟁목록에서 특정 전쟁 column들 업데이트하기
 - 전쟁발발기준에 대한 영향력을 확인하기 위한 평가지표 만들기
   - 전쟁발발시점 기업별 close price - open pirce 차이 비교
   - 발발당일, 발발 1일차, 발발 2일차 close price - open price 차이비교
   - 발발하루전~5일전까지 일별 close price - open price 차의 평균
   - index : 전쟁발발전 5일간 차이평균 대비 일변화량

In [67]:
import datetime

def get_days(yyyy,mm,dd):
    days = ['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일']
    return days[datetime.date(yyyy, mm, dd).weekday()]

start_war = ls_war2.loc[idx,"발발시점"]



yyyy = 2001
mm = 9
days = 11

print(f'{yyyy}년 {mm}일 {day}일은 {get_days(yyyy,mm,day)}입니다')

2001년 9일 11일은 화요일입니다


In [68]:
start_war.weekday()
# 0:월, 1:화, 2:수, 3:목, 4:금, 5:토, 6:일

1

In [436]:
# 선택된 전쟁을 기준으로 평가기준 계산을 위한 시점들 저장

idx=17
start_war = ls_war2.loc[idx,"발발시점"]
name_of_war = ls_war2.loc[idx,"갈등이름"]
cate_of_war = ls_war2.loc[idx,"갈등종류"]
counts_of_nations = ls_war2.loc[idx,"참전국수"]

a1 = ls_war2.loc[idx,"발발시점"] + timedelta(days=1)
a2 = ls_war2.loc[idx,"발발시점"] + timedelta(days=2)
a3 = ls_war2.loc[idx,"발발시점"] + timedelta(days=3)

b1 = ls_war2.loc[idx,"발발시점"] - timedelta(days=1)
b2 = ls_war2.loc[idx,"발발시점"] - timedelta(days=2)
b3 = ls_war2.loc[idx,"발발시점"] - timedelta(days=3)
b4 = ls_war2.loc[idx,"발발시점"] - timedelta(days=4)
b5 = ls_war2.loc[idx,"발발시점"] - timedelta(days=5)


df_3['Name_of_war'] = name_of_war
df_3['Start_date'] = start_war
df_3['Cate_of_war'] = cate_of_war
df_3['Count_of_nations'] = counts_of_nations

df_3.head(2)

,Symbol,Name,Industry,Name_of_war,Start_date,Cate_of_war,Count_of_nations
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",리비아내전,2011-02-15,내란,21
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,리비아내전,2011-02-15,내란,21


In [437]:
start_war

Timestamp('2011-02-15 00:00:00')

In [438]:
real_df.head(2)

,Unnamed: 0,Date,Close,Open,High,Low,Volume,Change,Symbol
0,0,2000-01-03,1.00,0.94,1.00,0.91,535820000.0,0.087,AAPL
1,1,2000-01-04,0.92,0.97,0.99,0.90,511730000.0,-0.080,AAPL


In [439]:
real_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358746 entries, 0 to 358745
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   Unnamed: 0  358746 non-null  int64         
 1   Date        358746 non-null  datetime64[ns]
 2   Close       358746 non-null  float64       
 3   Open        358746 non-null  float64       
 4   High        358746 non-null  float64       
 5   Low         358746 non-null  float64       
 6   Volume      358746 non-null  float64       
 7   Change      358746 non-null  float64       
 8   Symbol      358746 non-null  object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(1)
memory usage: 24.6+ MB


In [440]:
# 평가기준을 계산하여 저장 후 Dataframe 만들기
# 전쟁발발시점
start_war 

# 전쟁발발시점이 토요일일 경우
d_1_start = start_war + timedelta(days=2) #월요일
d_1_a1 = d_1_start + timedelta(days=1) #화요일
d_1_a2 = d_1_start + timedelta(days=2) #수요일

d_1_b1 = start_war - timedelta(days=1) #금요일
d_1_b2 = start_war - timedelta(days=2) #목요일
d_1_b3 = start_war - timedelta(days=3) #수요일
d_1_b4 = start_war - timedelta(days=4) #화요일
d_1_b5 = start_war - timedelta(days=5) #월요일

# 전쟁발발시점이 일요일일 경우

d_2_start = start_war + timedelta(days=1) #월요일
d_2_a1 = d_2_start + timedelta(days=1) #화요일
d_2_a2 = d_2_start + timedelta(days=2) #수요일

d_2_b1 = start_war - timedelta(days=2) #금요일
d_2_b2 = start_war - timedelta(days=3) #목요일
d_2_b3 = start_war - timedelta(days=4) #수요일
d_2_b4 = start_war - timedelta(days=5) #화요일
d_2_b5 = start_war - timedelta(days=6) #월요일


# 전쟁발발시점이 월요일 경우

d_3_start = start_war #월요일
d_3_a1 = d_3_start + timedelta(days=1) #화요일
d_3_a2 = d_3_start + timedelta(days=2) #수요일

d_3_b1 = start_war - timedelta(days=3) #금요일
d_3_b2 = start_war - timedelta(days=4) #목요일
d_3_b3 = start_war - timedelta(days=5) #수요일
d_3_b4 = start_war - timedelta(days=6) #화요일
d_3_b5 = start_war - timedelta(days=7) #월요일


# 전쟁발발시점이 화요일 경우

d_4_start = start_war #화요일
d_4_a1 = d_4_start + timedelta(days=1) #수요일
d_4_a2 = d_4_start + timedelta(days=2) #목요일

d_4_b1 = start_war - timedelta(days=1) #월요일
d_4_b2 = start_war - timedelta(days=4) #금요일
d_4_b3 = start_war - timedelta(days=5) #목요일
d_4_b4 = start_war - timedelta(days=6) #수요일
d_4_b5 = start_war - timedelta(days=7) #화요일


# 전쟁발발시점이 수요일 경우

d_5_start = start_war #수요일
d_5_a1 = d_5_start + timedelta(days=1) #목요일
d_5_a2 = d_5_start + timedelta(days=2) #금요일

d_5_b1 = start_war - timedelta(days=1) #화요일
d_5_b2 = start_war - timedelta(days=2) #월요일
d_5_b3 = start_war - timedelta(days=5) #금요일
d_5_b4 = start_war - timedelta(days=6) #목요일
d_5_b5 = start_war - timedelta(days=7) #수요일


# 전쟁발발시점이 목요일 경우

d_6_start = start_war #목요일
d_6_a1 = d_6_start + timedelta(days=1) #금요일
d_6_a2 = d_6_start + timedelta(days=4) #월요일

d_6_b1 = start_war - timedelta(days=1) #수요일
d_6_b2 = start_war - timedelta(days=2) #화요일
d_6_b3 = start_war - timedelta(days=3) #월요일
d_6_b4 = start_war - timedelta(days=6) #금요일
d_6_b5 = start_war - timedelta(days=7) #목요일


# 전쟁발발시점이 금요일 경우

d_7_start = start_war #금요일
d_7_a1 = d_7_start + timedelta(days=3) #월요일
d_7_a2 = d_7_start + timedelta(days=4) #화요일

d_7_b1 = start_war - timedelta(days=1) #목요일
d_7_b2 = start_war - timedelta(days=2) #수요일
d_7_b3 = start_war - timedelta(days=3) #화요일
d_7_b4 = start_war - timedelta(days=4) #월요일
d_7_b5 = start_war - timedelta(days=7) #금요일



# if start_war == 5: # startime이 토요일이면
if  start_war.weekday() == 5: 
    change_a1 = real_df[real_df['Date']==d_1_start]["Close"] - real_df[real_df['Date']==d_1_start]["Open"]
    change_a2 = real_df[real_df['Date']==d_1_a1]["Close"] - real_df[real_df['Date']==d_1_a1]["Open"]
    change_a3 = real_df[real_df['Date']==d_1_a2]["Close"] - real_df[real_df['Date']==d_1_a2]["Open"]
    change_b1 = real_df[real_df['Date']==d_1_b1]["Close"] - real_df[real_df['Date']==d_1_b1]["Open"]
    change_b2 = real_df[real_df['Date']==d_1_b2]["Close"] - real_df[real_df['Date']==d_1_b2]["Open"]
    change_b3 = real_df[real_df['Date']==d_1_b3]["Close"] - real_df[real_df['Date']==d_1_b3]["Open"]
    change_b4 = real_df[real_df['Date']==d_1_b4]["Close"] - real_df[real_df['Date']==d_1_b4]["Open"]
    change_b5 = real_df[real_df['Date']==d_1_b5]["Close"] - real_df[real_df['Date']==d_1_b5]["Open"]
    
elif start_war.weekday() == 6:    
    change_a1 = real_df[real_df['Date']==d_2_start]["Close"] - real_df[real_df['Date']==d_2_start]["Open"]
    change_a2 = real_df[real_df['Date']==d_2_a1]["Close"] - real_df[real_df['Date']==d_2_a1]["Open"]
    change_a3 = real_df[real_df['Date']==d_2_a2]["Close"] - real_df[real_df['Date']==d_2_a2]["Open"]
    change_b1 = real_df[real_df['Date']==d_2_b1]["Close"] - real_df[real_df['Date']==d_2_b1]["Open"]
    change_b2 = real_df[real_df['Date']==d_2_b2]["Close"] - real_df[real_df['Date']==d_2_b2]["Open"]
    change_b3 = real_df[real_df['Date']==d_2_b3]["Close"] - real_df[real_df['Date']==d_2_b3]["Open"]
    change_b4 = real_df[real_df['Date']==d_2_b4]["Close"] - real_df[real_df['Date']==d_2_b4]["Open"]
    change_b5 = real_df[real_df['Date']==d_2_b5]["Close"] - real_df[real_df['Date']==d_2_b5]["Open"]

    
elif start_war.weekday() == 0:    
    change_a1 = real_df[real_df['Date']==d_3_start]["Close"] - real_df[real_df['Date']==d_3_start]["Open"]
    change_a2 = real_df[real_df['Date']==d_3_a1]["Close"] - real_df[real_df['Date']==d_3_a1]["Open"]
    change_a3 = real_df[real_df['Date']==d_3_a2]["Close"] - real_df[real_df['Date']==d_3_a2]["Open"]
    change_b1 = real_df[real_df['Date']==d_3_b1]["Close"] - real_df[real_df['Date']==d_3_b1]["Open"]
    change_b2 = real_df[real_df['Date']==d_3_b2]["Close"] - real_df[real_df['Date']==d_3_b2]["Open"]
    change_b3 = real_df[real_df['Date']==d_3_b3]["Close"] - real_df[real_df['Date']==d_3_b3]["Open"]
    change_b4 = real_df[real_df['Date']==d_3_b4]["Close"] - real_df[real_df['Date']==d_3_b4]["Open"]
    change_b5 = real_df[real_df['Date']==d_3_b5]["Close"] - real_df[real_df['Date']==d_3_b5]["Open"]    
    

elif start_war.weekday() == 1:    
    change_a1 = real_df[real_df['Date']==d_4_start]["Close"] - real_df[real_df['Date']==d_4_start]["Open"]
    change_a2 = real_df[real_df['Date']==d_4_a1]["Close"] - real_df[real_df['Date']==d_4_a1]["Open"]
    change_a3 = real_df[real_df['Date']==d_4_a2]["Close"] - real_df[real_df['Date']==d_4_a2]["Open"]
    change_b1 = real_df[real_df['Date']==d_4_b1]["Close"] - real_df[real_df['Date']==d_4_b1]["Open"]
    change_b2 = real_df[real_df['Date']==d_4_b2]["Close"] - real_df[real_df['Date']==d_4_b2]["Open"]
    change_b3 = real_df[real_df['Date']==d_4_b3]["Close"] - real_df[real_df['Date']==d_4_b3]["Open"]
    change_b4 = real_df[real_df['Date']==d_4_b4]["Close"] - real_df[real_df['Date']==d_4_b4]["Open"]
    change_b5 = real_df[real_df['Date']==d_4_b5]["Close"] - real_df[real_df['Date']==d_4_b5]["Open"]       

    
elif start_war.weekday() == 2:    
    change_a1 = real_df[real_df['Date']==d_5_start]["Close"] - real_df[real_df['Date']==d_5_start]["Open"]
    change_a2 = real_df[real_df['Date']==d_5_a1]["Close"] - real_df[real_df['Date']==d_5_a1]["Open"]
    change_a3 = real_df[real_df['Date']==d_5_a2]["Close"] - real_df[real_df['Date']==d_5_a2]["Open"]
    change_b1 = real_df[real_df['Date']==d_5_b1]["Close"] - real_df[real_df['Date']==d_5_b1]["Open"]
    change_b2 = real_df[real_df['Date']==d_5_b2]["Close"] - real_df[real_df['Date']==d_5_b2]["Open"]
    change_b3 = real_df[real_df['Date']==d_5_b3]["Close"] - real_df[real_df['Date']==d_5_b3]["Open"]
    change_b4 = real_df[real_df['Date']==d_5_b4]["Close"] - real_df[real_df['Date']==d_5_b4]["Open"]
    change_b5 = real_df[real_df['Date']==d_5_b5]["Close"] - real_df[real_df['Date']==d_5_b5]["Open"]        

elif start_war.weekday() == 3:    
    change_a1 = real_df[real_df['Date']==d_6_start]["Close"] - real_df[real_df['Date']==d_6_start]["Open"]
    change_a2 = real_df[real_df['Date']==d_6_a1]["Close"] - real_df[real_df['Date']==d_6_a1]["Open"]
    change_a3 = real_df[real_df['Date']==d_6_a2]["Close"] - real_df[real_df['Date']==d_6_a2]["Open"]
    change_b1 = real_df[real_df['Date']==d_6_b1]["Close"] - real_df[real_df['Date']==d_6_b1]["Open"]
    change_b2 = real_df[real_df['Date']==d_6_b2]["Close"] - real_df[real_df['Date']==d_6_b2]["Open"]
    change_b3 = real_df[real_df['Date']==d_6_b3]["Close"] - real_df[real_df['Date']==d_6_b3]["Open"]
    change_b4 = real_df[real_df['Date']==d_6_b4]["Close"] - real_df[real_df['Date']==d_6_b4]["Open"]
    change_b5 = real_df[real_df['Date']==d_6_b5]["Close"] - real_df[real_df['Date']==d_6_b5]["Open"]      
        
    
else:          
    change_a1 = real_df[real_df['Date']==d_7_start]["Close"] - real_df[real_df['Date']==d_7_start]["Open"]
    change_a2 = real_df[real_df['Date']==d_7_a1]["Close"] - real_df[real_df['Date']==d_7_a1]["Open"]
    change_a3 = real_df[real_df['Date']==d_7_a2]["Close"] - real_df[real_df['Date']==d_7_a2]["Open"]
    change_b1 = real_df[real_df['Date']==d_7_b1]["Close"] - real_df[real_df['Date']==d_7_b1]["Open"]
    change_b2 = real_df[real_df['Date']==d_7_b2]["Close"] - real_df[real_df['Date']==d_7_b2]["Open"]
    change_b3 = real_df[real_df['Date']==d_7_b3]["Close"] - real_df[real_df['Date']==d_7_b3]["Open"]
    change_b4 = real_df[real_df['Date']==d_7_b4]["Close"] - real_df[real_df['Date']==d_7_b4]["Open"]
    change_b5 = real_df[real_df['Date']==d_7_b5]["Close"] - real_df[real_df['Date']==d_7_b5]["Open"]  


In [441]:
# 전쟁발발 첫날, 2일차, 3일차 가격

In [442]:
if  start_war.weekday() == 5: 
    df_change_a1 = [real_df[real_df['Date']==d_1_start]["Symbol"],change_a1]
    df_change_a1 = pd.DataFrame(df_change_a1)
    df_change_a1 = df_change_a1.transpose()
    df_change_a1.rename(columns={"Unnamed 0" : "change_a1"}, inplace=True)
    
    df_change_a2 = [real_df[real_df['Date']==d_1_a1]["Symbol"],change_a2]
    df_change_a2 = pd.DataFrame(df_change_a2)
    df_change_a2 = df_change_a2.transpose()
    df_change_a2.rename(columns={"Unnamed 0" : "change_a2"}, inplace=True)    
    
    df_change_a3 = [real_df[real_df['Date']==d_1_a2]["Symbol"],change_a3]
    df_change_a3 = pd.DataFrame(df_change_a3)
    df_change_a3 = df_change_a3.transpose()
    df_change_a3.rename(columns={"Unnamed 0" : "change_a3"}, inplace=True)   
    
elif start_war.weekday() == 6:  
    df_change_a1 = [real_df[real_df['Date']==d_2_start]["Symbol"],change_a1]
    df_change_a1 = pd.DataFrame(df_change_a1)
    df_change_a1 = df_change_a1.transpose()
    df_change_a1.rename(columns={"Unnamed 0" : "change_a1"}, inplace=True)
    
    df_change_a2 = [real_df[real_df['Date']==d_2_a1]["Symbol"],change_a2]
    df_change_a2 = pd.DataFrame(df_change_a2)
    df_change_a2 = df_change_a2.transpose()
    df_change_a2.rename(columns={"Unnamed 0" : "change_a2"}, inplace=True)    
    
    df_change_a3 = [real_df[real_df['Date']==d_2_a2]["Symbol"],change_a3]
    df_change_a3 = pd.DataFrame(df_change_a3)
    df_change_a3 = df_change_a3.transpose()
    df_change_a3.rename(columns={"Unnamed 0" : "change_a3"}, inplace=True)   

elif start_war.weekday() == 0:  
    df_change_a1 = [real_df[real_df['Date']==d_3_start]["Symbol"],change_a1]
    df_change_a1 = pd.DataFrame(df_change_a1)
    df_change_a1 = df_change_a1.transpose()
    df_change_a1.rename(columns={"Unnamed 0" : "change_a1"}, inplace=True)
    
    df_change_a2 = [real_df[real_df['Date']==d_3_a1]["Symbol"],change_a2]
    df_change_a2 = pd.DataFrame(df_change_a2)
    df_change_a2 = df_change_a2.transpose()
    df_change_a2.rename(columns={"Unnamed 0" : "change_a2"}, inplace=True)    
    
    df_change_a3 = [real_df[real_df['Date']==d_3_a2]["Symbol"],change_a3]
    df_change_a3 = pd.DataFrame(df_change_a3)
    df_change_a3 = df_change_a3.transpose()
    df_change_a3.rename(columns={"Unnamed 0" : "change_a3"}, inplace=True)   
    
elif start_war.weekday() == 1:  
    df_change_a1 = [real_df[real_df['Date']==d_4_start]["Symbol"],change_a1]
    df_change_a1 = pd.DataFrame(df_change_a1)
    df_change_a1 = df_change_a1.transpose()
    df_change_a1.rename(columns={"Unnamed 0" : "change_a1"}, inplace=True)
    
    df_change_a2 = [real_df[real_df['Date']==d_4_a1]["Symbol"],change_a2]
    df_change_a2 = pd.DataFrame(df_change_a2)
    df_change_a2 = df_change_a2.transpose()
    df_change_a2.rename(columns={"Unnamed 0" : "change_a2"}, inplace=True)    
    
    df_change_a3 = [real_df[real_df['Date']==d_4_a2]["Symbol"],change_a3]
    df_change_a3 = pd.DataFrame(df_change_a3)
    df_change_a3 = df_change_a3.transpose()
    df_change_a3.rename(columns={"Unnamed 0" : "change_a3"}, inplace=True)   

elif start_war.weekday() == 2:  
    df_change_a1 = [real_df[real_df['Date']==d_5_start]["Symbol"],change_a1]
    df_change_a1 = pd.DataFrame(df_change_a1)
    df_change_a1 = df_change_a1.transpose()
    df_change_a1.rename(columns={"Unnamed 0" : "change_a1"}, inplace=True)
    
    df_change_a2 = [real_df[real_df['Date']==d_5_a1]["Symbol"],change_a2]
    df_change_a2 = pd.DataFrame(df_change_a2)
    df_change_a2 = df_change_a2.transpose()
    df_change_a2.rename(columns={"Unnamed 0" : "change_a2"}, inplace=True)    
    
    df_change_a3 = [real_df[real_df['Date']==d_5_a2]["Symbol"],change_a3]
    df_change_a3 = pd.DataFrame(df_change_a3)
    df_change_a3 = df_change_a3.transpose()
    df_change_a3.rename(columns={"Unnamed 0" : "change_a3"}, inplace=True)  

elif start_war.weekday() == 3:  
    df_change_a1 = [real_df[real_df['Date']==d_6_start]["Symbol"],change_a1]
    df_change_a1 = pd.DataFrame(df_change_a1)
    df_change_a1 = df_change_a1.transpose()
    df_change_a1.rename(columns={"Unnamed 0" : "change_a1"}, inplace=True)
    
    df_change_a2 = [real_df[real_df['Date']==d_6_a1]["Symbol"],change_a2]
    df_change_a2 = pd.DataFrame(df_change_a2)
    df_change_a2 = df_change_a2.transpose()
    df_change_a2.rename(columns={"Unnamed 0" : "change_a2"}, inplace=True)    
    
    df_change_a3 = [real_df[real_df['Date']==d_6_a2]["Symbol"],change_a3]
    df_change_a3 = pd.DataFrame(df_change_a3)
    df_change_a3 = df_change_a3.transpose()
    df_change_a3.rename(columns={"Unnamed 0" : "change_a3"}, inplace=True) 
    
else :  
    df_change_a1 = [real_df[real_df['Date']==d_7_start]["Symbol"],change_a1]
    df_change_a1 = pd.DataFrame(df_change_a1)
    df_change_a1 = df_change_a1.transpose()
    df_change_a1.rename(columns={"Unnamed 0" : "change_a1"}, inplace=True)
    
    df_change_a2 = [real_df[real_df['Date']==d_7_a1]["Symbol"],change_a2]
    df_change_a2 = pd.DataFrame(df_change_a2)
    df_change_a2 = df_change_a2.transpose()
    df_change_a2.rename(columns={"Unnamed 0" : "change_a2"}, inplace=True)    
    
    df_change_a3 = [real_df[real_df['Date']==d_7_a2]["Symbol"],change_a3]
    df_change_a3 = pd.DataFrame(df_change_a3)
    df_change_a3 = df_change_a3.transpose()
    df_change_a3.rename(columns={"Unnamed 0" : "change_a3"}, inplace=True) 

In [443]:
#전쟁발발전 5일전 가격

In [444]:
if  start_war.weekday() == 5: 
    df_change_b1 = [real_df[real_df['Date']==d_1_b1]["Symbol"],change_b1]
    df_change_b1 = pd.DataFrame(df_change_b1)
    df_change_b1 = df_change_b1.transpose()
    df_change_b1.rename(columns={"Unnamed 0" : "change_b1"}, inplace=True)
    
    df_change_b2 = [real_df[real_df['Date']==d_1_b2]["Symbol"],change_b2]
    df_change_b2 = pd.DataFrame(df_change_b2)
    df_change_b2 = df_change_b2.transpose()
    df_change_b2.rename(columns={"Unnamed 0" : "change_b2"}, inplace=True)    
    
    df_change_b3 = [real_df[real_df['Date']==d_1_b3]["Symbol"],change_b3]
    df_change_b3 = pd.DataFrame(df_change_b3)
    df_change_b3 = df_change_b3.transpose()
    df_change_b3.rename(columns={"Unnamed 0" : "change_b3"}, inplace=True) 
    
    
    df_change_b4 = [real_df[real_df['Date']==d_1_b4]["Symbol"],change_b4]
    df_change_b4 = pd.DataFrame(df_change_b4)
    df_change_b4 = df_change_b4.transpose()
    df_change_b4.rename(columns={"Unnamed 0" : "change_b4"}, inplace=True)    
    
    df_change_b5 = [real_df[real_df['Date']==d_1_b5]["Symbol"],change_b5]
    df_change_b5 = pd.DataFrame(df_change_b5)
    df_change_b5 = df_change_b5.transpose()
    df_change_b5.rename(columns={"Unnamed 0" : "change_b5"}, inplace=True)    
    
elif start_war.weekday() == 6:  
    df_change_b1 = [real_df[real_df['Date']==d_2_b1]["Symbol"],change_b1]
    df_change_b1 = pd.DataFrame(df_change_b1)
    df_change_b1 = df_change_b1.transpose()
    df_change_b1.rename(columns={"Unnamed 0" : "change_b1"}, inplace=True)
    
    df_change_b2 = [real_df[real_df['Date']==d_2_b2]["Symbol"],change_b2]
    df_change_b2 = pd.DataFrame(df_change_b2)
    df_change_b2 = df_change_b2.transpose()
    df_change_b2.rename(columns={"Unnamed 0" : "change_b2"}, inplace=True)    
    
    df_change_b3 = [real_df[real_df['Date']==d_2_b3]["Symbol"],change_b3]
    df_change_b3 = pd.DataFrame(df_change_b3)
    df_change_b3 = df_change_b3.transpose()
    df_change_b3.rename(columns={"Unnamed 0" : "change_b3"}, inplace=True) 
    
    
    df_change_b4 = [real_df[real_df['Date']==d_2_b4]["Symbol"],change_b4]
    df_change_b4 = pd.DataFrame(df_change_b4)
    df_change_b4 = df_change_b4.transpose()
    df_change_b4.rename(columns={"Unnamed 0" : "change_b4"}, inplace=True)    
    
    df_change_b5 = [real_df[real_df['Date']==d_2_b5]["Symbol"],change_b5]
    df_change_b5 = pd.DataFrame(df_change_b5)
    df_change_b5 = df_change_b5.transpose()
    df_change_b5.rename(columns={"Unnamed 0" : "change_b5"}, inplace=True)   

elif start_war.weekday() == 0:  
    df_change_b1 = [real_df[real_df['Date']==d_3_b1]["Symbol"],change_b1]
    df_change_b1 = pd.DataFrame(df_change_b1)
    df_change_b1 = df_change_b1.transpose()
    df_change_b1.rename(columns={"Unnamed 0" : "change_b1"}, inplace=True)
    
    df_change_b2 = [real_df[real_df['Date']==d_3_b2]["Symbol"],change_b2]
    df_change_b2 = pd.DataFrame(df_change_b2)
    df_change_b2 = df_change_b2.transpose()
    df_change_b2.rename(columns={"Unnamed 0" : "change_b2"}, inplace=True)    
    
    df_change_b3 = [real_df[real_df['Date']==d_3_b3]["Symbol"],change_b3]
    df_change_b3 = pd.DataFrame(df_change_b3)
    df_change_b3 = df_change_b3.transpose()
    df_change_b3.rename(columns={"Unnamed 0" : "change_b3"}, inplace=True) 
    
    
    df_change_b4 = [real_df[real_df['Date']==d_3_b4]["Symbol"],change_b4]
    df_change_b4 = pd.DataFrame(df_change_b4)
    df_change_b4 = df_change_b4.transpose()
    df_change_b4.rename(columns={"Unnamed 0" : "change_b4"}, inplace=True)    
    
    df_change_b5 = [real_df[real_df['Date']==d_3_b5]["Symbol"],change_b5]
    df_change_b5 = pd.DataFrame(df_change_b5)
    df_change_b5 = df_change_b5.transpose()
    df_change_b5.rename(columns={"Unnamed 0" : "change_b5"}, inplace=True)     
    
elif start_war.weekday() == 1:  
    df_change_b1 = [real_df[real_df['Date']==d_4_b1]["Symbol"],change_b1]
    df_change_b1 = pd.DataFrame(df_change_b1)
    df_change_b1 = df_change_b1.transpose()
    df_change_b1.rename(columns={"Unnamed 0" : "change_b1"}, inplace=True)
    
    df_change_b2 = [real_df[real_df['Date']==d_4_b2]["Symbol"],change_b2]
    df_change_b2 = pd.DataFrame(df_change_b2)
    df_change_b2 = df_change_b2.transpose()
    df_change_b2.rename(columns={"Unnamed 0" : "change_b2"}, inplace=True)    
    
    df_change_b3 = [real_df[real_df['Date']==d_4_b3]["Symbol"],change_b3]
    df_change_b3 = pd.DataFrame(df_change_b3)
    df_change_b3 = df_change_b3.transpose()
    df_change_b3.rename(columns={"Unnamed 0" : "change_b3"}, inplace=True) 
    
    
    df_change_b4 = [real_df[real_df['Date']==d_4_b4]["Symbol"],change_b4]
    df_change_b4 = pd.DataFrame(df_change_b4)
    df_change_b4 = df_change_b4.transpose()
    df_change_b4.rename(columns={"Unnamed 0" : "change_b4"}, inplace=True)    
    
    df_change_b5 = [real_df[real_df['Date']==d_4_b5]["Symbol"],change_b5]
    df_change_b5 = pd.DataFrame(df_change_b5)
    df_change_b5 = df_change_b5.transpose()
    df_change_b5.rename(columns={"Unnamed 0" : "change_b5"}, inplace=True)   

elif start_war.weekday() == 2:  
    df_change_b1 = [real_df[real_df['Date']==d_5_b1]["Symbol"],change_b1]
    df_change_b1 = pd.DataFrame(df_change_b1)
    df_change_b1 = df_change_b1.transpose()
    df_change_b1.rename(columns={"Unnamed 0" : "change_b1"}, inplace=True)
    
    df_change_b2 = [real_df[real_df['Date']==d_5_b2]["Symbol"],change_b2]
    df_change_b2 = pd.DataFrame(df_change_b2)
    df_change_b2 = df_change_b2.transpose()
    df_change_b2.rename(columns={"Unnamed 0" : "change_b2"}, inplace=True)    
    
    df_change_b3 = [real_df[real_df['Date']==d_5_b3]["Symbol"],change_b3]
    df_change_b3 = pd.DataFrame(df_change_b3)
    df_change_b3 = df_change_b3.transpose()
    df_change_b3.rename(columns={"Unnamed 0" : "change_b3"}, inplace=True) 
    
    
    df_change_b4 = [real_df[real_df['Date']==d_5_b4]["Symbol"],change_b4]
    df_change_b4 = pd.DataFrame(df_change_b4)
    df_change_b4 = df_change_b4.transpose()
    df_change_b4.rename(columns={"Unnamed 0" : "change_b4"}, inplace=True)    
    
    df_change_b5 = [real_df[real_df['Date']==d_5_b5]["Symbol"],change_b5]
    df_change_b5 = pd.DataFrame(df_change_b5)
    df_change_b5 = df_change_b5.transpose()
    df_change_b5.rename(columns={"Unnamed 0" : "change_b5"}, inplace=True)  

elif start_war.weekday() == 3:  
    df_change_b1 = [real_df[real_df['Date']==d_6_b1]["Symbol"],change_b1]
    df_change_b1 = pd.DataFrame(df_change_b1)
    df_change_b1 = df_change_b1.transpose()
    df_change_b1.rename(columns={"Unnamed 0" : "change_b1"}, inplace=True)
    
    df_change_b2 = [real_df[real_df['Date']==d_6_b2]["Symbol"],change_b2]
    df_change_b2 = pd.DataFrame(df_change_b2)
    df_change_b2 = df_change_b2.transpose()
    df_change_b2.rename(columns={"Unnamed 0" : "change_b2"}, inplace=True)    
    
    df_change_b3 = [real_df[real_df['Date']==d_6_b3]["Symbol"],change_b3]
    df_change_b3 = pd.DataFrame(df_change_b3)
    df_change_b3 = df_change_b3.transpose()
    df_change_b3.rename(columns={"Unnamed 0" : "change_b3"}, inplace=True)  
    
    df_change_b4 = [real_df[real_df['Date']==d_6_b4]["Symbol"],change_b4]
    df_change_b4 = pd.DataFrame(df_change_b4)
    df_change_b4 = df_change_b4.transpose()
    df_change_b4.rename(columns={"Unnamed 0" : "change_b4"}, inplace=True)    
    
    df_change_b5 = [real_df[real_df['Date']==d_6_b5]["Symbol"],change_b5]
    df_change_b5 = pd.DataFrame(df_change_b5)
    df_change_b5 = df_change_b5.transpose()
    df_change_b5.rename(columns={"Unnamed 0" : "change_b5"}, inplace=True)   
    
else :  
    df_change_b1 = [real_df[real_df['Date']==d_7_b1]["Symbol"],change_b1]
    df_change_b1 = pd.DataFrame(df_change_b1)
    df_change_b1 = df_change_b1.transpose()
    df_change_b1.rename(columns={"Unnamed 0" : "change_b1"}, inplace=True)
    
    df_change_b2 = [real_df[real_df['Date']==d_7_b2]["Symbol"],change_b2]
    df_change_b2 = pd.DataFrame(df_change_b2)
    df_change_b2 = df_change_b2.transpose()
    df_change_b2.rename(columns={"Unnamed 0" : "change_b2"}, inplace=True)    
    
    df_change_b3 = [real_df[real_df['Date']==d_7_b3]["Symbol"],change_b3]
    df_change_b3 = pd.DataFrame(df_change_b3)
    df_change_b3 = df_change_b3.transpose()
    df_change_b3.rename(columns={"Unnamed 0" : "change_b3"}, inplace=True) 
    
    
    df_change_b4 = [real_df[real_df['Date']==d_7_b4]["Symbol"],change_b4]
    df_change_b4 = pd.DataFrame(df_change_b4)
    df_change_b4 = df_change_b4.transpose()
    df_change_b4.rename(columns={"Unnamed 0" : "change_b4"}, inplace=True)    
    
    df_change_b5 = [real_df[real_df['Date']==d_7_b5]["Symbol"],change_b5]
    df_change_b5 = pd.DataFrame(df_change_b5)
    df_change_b5 = df_change_b5.transpose()
    df_change_b5.rename(columns={"Unnamed 0" : "change_b5"}, inplace=True)    

In [445]:
merged_df = pd.merge(df_change_a1,df_change_a2)
merged_df = pd.merge(merged_df, df_change_a3)
merged_df = pd.merge(merged_df, df_change_b1)
merged_df = pd.merge(merged_df, df_change_b2)
merged_df = pd.merge(merged_df, df_change_b3)
merged_df = pd.merge(merged_df, df_change_b4)
merged_df = pd.merge(merged_df, df_change_b5)

In [446]:
merged_df.tail(2)

,Symbol,change_a1,change_a2,change_a3,change_b1,change_b2,change_b3,change_b4,change_b5
65,WFC,-0.26,-0.41,-0.11,0.24,1.07,-0.07,-0.1,0.81
66,DHR,0.26,-0.24,0.17,-0.13,0.9,1.34,0.35,0.15


In [447]:
# 기업별 symbol자료와 평가지표들 Merge하여 최종 Dataframe 만들기

df_result = pd.merge(df_3, merged_df)
df_result["avg.change"] = (df_result["change_b1"]+df_result["change_b1"]+df_result["change_b3"]
                           +df_result["change_b4"]+df_result["change_b5"])/5
df_result["index_d1"] = df_result["change_a1"] / df_result["avg.change"] * 100
df_result["index_d2"] = df_result["change_a2"] / df_result["avg.change"] * 100
df_result["index_d3"] = df_result["change_a3"] / df_result["avg.change"] * 100

df_result
show_df = df_result.drop(["Start_date", "Cate_of_war", "Count_of_nations", "change_b3","change_b4","change_b5"], axis=1)
show_df.head(10)

,Symbol,Name,Industry,Name_of_war,change_a1,change_a2,change_a3,change_b1,change_b2,avg.change,index_d1,index_d2,index_d3
0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",리비아내전,0.02,0.08,0.04,0.09,0.07,0.048,41.666667,166.666667,83.333333
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,리비아내전,-0.08,-0.03,0.24,0.03,-0.51,-0.082,97.560976,36.585366,-292.682927
2,AMZN,Amazon.com Inc,다양한 소매업,리비아내전,0.3,-3.15,1.99,1.17,3.69,2.542,11.801731,-123.918175,78.284815
3,GOOGL,Alphabet Inc Class A,소프트웨어 및 IT서비스,리비아내전,-1.58,-0.71,2.0,2.41,5.36,1.528,-103.403141,-46.465969,130.890052
4,TSLA,Tesla Inc,자동차 및 자동차 부품,리비아내전,-0.03,0.33,-0.21,-0.11,0.0,-0.056,53.571429,-589.285714,375.0
5,NVDA,NVIDIA Corp,반도체 및 반도체 장비,리비아내전,-0.6,0.53,2.96,-0.64,0.73,-0.532,112.781955,-99.62406,-556.390977
6,INTC,Intel Corp,반도체 및 반도체 장비,리비아내전,-0.03,0.18,0.29,-0.06,0.13,0.034,-88.235294,529.411765,852.941176
7,ASML,ASML Holding NV ADR,반도체 및 반도체 장비,리비아내전,-0.21,0.67,0.72,1.01,0.44,0.492,-42.682927,136.178862,146.341463
8,CMCSA,Comcast Corp Class A,미디어 및 출판,리비아내전,0.38,0.2,0.16,-0.06,0.34,0.054,703.703704,370.37037,296.296296
9,NFLX,Netflix Inc,소프트웨어 및 IT서비스,리비아내전,-0.38,-0.71,-0.04,1.69,0.98,0.866,-43.879908,-81.986143,-4.618938


In [435]:
show_df.to_csv("datas/show_df_9.csv")

In [453]:
df_0=pd.read_csv("datas/show_df_0.csv")
df_1=pd.read_csv("datas/show_df_1.csv")
df_2=pd.read_csv("datas/show_df_2.csv")
df_3=pd.read_csv("datas/show_df_3.csv")
df_4=pd.read_csv("datas/show_df_4.csv")
df_5=pd.read_csv("datas/show_df_5.csv")
df_6=pd.read_csv("datas/show_df_6.csv")
df_8=pd.read_csv("datas/show_df_8.csv")
df_9=pd.read_csv("datas/show_df_9.csv")

In [457]:
df = pd.DataFrame()
df = pd.concat([df,df_0])
df = pd.concat([df,df_1])
df = pd.concat([df,df_2])
df = pd.concat([df,df_3])
df = pd.concat([df,df_4])
df = pd.concat([df,df_5])
df = pd.concat([df,df_6])
df = pd.concat([df,df_8])
df = pd.concat([df,df_9])
df

,Unnamed: 0,Symbol,Name,Industry,Name_of_war,change_a1,change_a2,change_a3,change_b1,change_b2,avg.change,index_d1,index_d2,index_d3
0,0,AAPL,Apple Inc,"컴퓨터, 전화 및 가전제품",영원한자유작전-필리핀,0.01,-0.01,0.01,0.00,0.00,-0.006,-166.666667,166.666667,-166.666667
1,1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,영원한자유작전-필리핀,0.44,-0.48,0.66,0.08,-0.46,0.080,550.000000,-600.000000,825.000000
2,2,AMZN,Amazon.com Inc,다양한 소매업,영원한자유작전-필리핀,-0.05,-0.88,-0.08,-0.69,0.00,-0.570,8.771930,154.385965,14.035088
3,3,NVDA,NVIDIA Corp,반도체 및 반도체 장비,영원한자유작전-필리핀,1.03,-0.55,0.04,-1.06,-1.12,-0.572,-180.069930,96.153846,-6.993007
4,4,INTC,Intel Corp,반도체 및 반도체 장비,영원한자유작전-필리핀,-0.26,-0.25,0.48,0.54,-0.27,-0.012,2166.666667,2083.333333,-4000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,40,LLY,Eli Lilly and Co,제약,킹스톤불안,-0.03,0.12,-0.73,0.12,-0.11,0.008,-375.000000,1500.000000,-9125.000000
41,41,MCD,Mcdonald's Corp,호텔 및 엔터테인먼트,킹스톤불안,-0.18,1.25,-1.88,0.79,-0.88,0.226,-79.646018,553.097345,-831.858407
42,42,DHR,Danaher Corp,헬스케어 장비 및 용품,킹스톤불안,-0.30,0.92,-0.43,1.21,-0.26,0.032,-937.500000,2875.000000,-1343.750000
43,43,WFC,Wells Fargo & Co,은행,킹스톤불안,-0.86,1.19,-1.19,2.06,-0.75,0.490,-175.510204,242.857143,-242.857143


In [462]:
import xlsxwriter

In [464]:
df.to_excel("datas/df.xls", sheet_name="sheet", engine="xlsxwriter", encoding="utf-8-sig")

In [ ]:
df = pd.DataFrame()
for idx in range(0, 9):
    symbol = us_cab.loc[idx, "Symbol"]
    globals()['df{}'.format(idx)] = fdr.DataReader(symbol,'2000-01-01', '2021-03-28')
    globals()['df{}'.format(idx)]["Symbol"] = symbol
    globals()['df{}'.format(idx)].reset_index(inplace = True)
    real_df = pd.concat([real_df, globals()['df{}'.format(idx)]], axis =0)
real_df.reset_index(inplace = True, drop = True)

### 3. index별로 필터링 해보기
- 발발이후 평균대비 일변화량 인덱스를 200으로 임의로 선택하여 필터링 시도
- 인덱스에 대한 이해
 - 200이상의 값은 전쟁발발시점 평기대비 변화의 폭 크다는 의미
 - `-`의 경우 오름세가 내림세로 바뀌거나 내림세가 오름세로 바뀐 경우
 - `+`의 경우 오름세에서 더올랐거나 내림세에서 더 내린 경우
 

In [258]:
#개전첫날 가격변동폭이 큰 기업

In [259]:
show_df[abs(show_df["index_d1"])>200]

,Symbol,Name,Industry,Name_of_war,change_a1,change_a2,change_a3,change_b1,change_b2,avg.change,index_d1,index_d2,index_d3
1,MSFT,Microsoft Corp,소프트웨어 및 IT서비스,영원한자유작전-필리핀,0.44,-0.48,0.66,0.08,-0.46,0.08,550.0,-600.0,825.0
4,INTC,Intel Corp,반도체 및 반도체 장비,영원한자유작전-필리핀,-0.26,-0.25,0.48,0.54,-0.27,-0.012,2166.666667,2083.333333,-4000.0
7,ADBE,Adobe Inc,소프트웨어 및 IT서비스,영원한자유작전-필리핀,0.39,-0.34,0.25,-0.06,-0.21,-0.038,-1026.315789,894.736842,-657.894737
12,JPM,JPMorgan Chase & Co,은행,영원한자유작전-필리핀,0.56,-0.99,-0.15,-0.34,-0.91,-0.096,-583.333333,1031.25,156.25
14,WMT,Walmart Inc,식품 및 약품소매,영원한자유작전-필리핀,1.11,-0.88,0.91,-0.04,-0.85,-0.016,-6937.5,5500.0,-5687.5
16,PG,Procter & Gamble Co,"개인, 가정 용품 및 서비스",영원한자유작전-필리핀,0.55,0.17,0.1,0.52,-0.24,0.204,269.607843,83.333333,49.019608
18,BAC,Bank of America Corp,은행,영원한자유작전-필리핀,0.33,0.1,0.04,-0.09,-0.67,-0.144,-229.166667,-69.444444,-27.777778
21,XOM,Exxon Mobil Corp,오일 및 가스,영원한자유작전-필리핀,0.41,-0.5,0.02,0.04,-0.89,-0.066,-621.212121,757.575758,-30.30303
25,ABT,Abbott Laboratories,헬스케어 장비 및 용품,영원한자유작전-필리핀,-0.21,0.36,0.4,-0.05,0.33,-0.042,500.0,-857.142857,-952.380952
26,NKE,Nike Inc Class B,섬유 및 의류,영원한자유작전-필리핀,0.13,-0.13,0.13,-0.09,-0.13,-0.014,-928.571429,928.571429,-928.571429


In [243]:
#개전2일차 가격변동폭이 큰 기업

In [226]:
show_df[abs(show_df["index_d2"])>200]

,Symbol,Name,Industry,Name_of_war,change_a1,change_a2,change_a3,change_b1,change_b2,avg.change,index_d1,index_d2,index_d3


In [227]:
#개전3일차 가격변동폭이 큰 기업

In [151]:
show_df[abs(show_df["index_d3"])>200]

,Symbol,Name,Industry,Name_of_war,change_a1,change_a2,change_a3,change_b1,change_b2,avg.change,index_d1,index_d2,index_d3


### 4. 현재까지 작업에서 찾은 문제들
- 전쟁발발시점을 기준으로 일주가 변화량을 탐색하는데 있어서 휴일,공휴일이 제외 필요
- 반복문으로 어떻게 구현할지

### 보강할 내용
- 날짜의 경우 발발시점 기준 5개의 날짜 데이트를 가져오도록 설정
- 마찬찬가로 발발시점 기준 3개의 날짜 데이트를 가져오도록 설정
- 금, 주가지수, 원유, 특정섹터로 등을 넣어봐라
- 그리고 전쟁목록 데이터를 좀더 늘려서
  전쟁규모 등과 금, 주가, 특정섹터 등의 자료와 상관관계를 분석해봐라
- 각각의 특성을 토대로 그래프등을 그려보는 것도 필요.